# How to transform depedency tree nodes as input sequences

In [1]:
import sys
sys.path.append('..')

In [2]:
from pad_sequences import pad_sequences_adjacency

import torch
print(torch.__version__)

import spacy

1.6.0


In [3]:
%%capture
!python -m spacy download de_core_news_sm

## Simple example
Assume an adjacency matrix is defined by index pairs `idx_pairs` and the matrix dimension `n_seqlen`

In [4]:
idx_pairs = [(0, 1), (2, 1), (3, 4), (5, 4), (7, 4), (11, 4), (6, 5), (10, 7), (8, 10), (9, 10)]
n_seqlen = 12

In [5]:
# torch
mat = torch.sparse.FloatTensor(
    torch.LongTensor(idx_pairs).t(),
    torch.FloatTensor([999 for _ in range(len(idx_pairs))]),
    torch.Size([n_seqlen, n_seqlen])
)
mat.to_dense()

tensor([[  0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])

## 

## Dependency Relations as Input Sequence

In [6]:
data = [
    "Die Freisetzung eines Arzneistoffs aus der Arzneiform steht fast vollständig unter pharmazeutisch-technologischer Kontrolle (ist also abhängig von der Galenik und der Formulierung der Arzneiform).",
    "Somit ist die Liberation der Arzneiform nur bedingt dem Bereich der Pharmakokinetik zuzuordnen.",
    "Alle weiteren Vorgänge sind physiologisch gesteuert und in weiten Grenzen variabel.",
    "Freisetzung ist die Grundvoraussetzung für die Wirkung eines Arzneimittels.",
]

In [7]:
import de_core_news_sm
nlp = de_core_news_sm.load()
nlp.disable_pipes(["ner", "tagger"])

# process texts
%time docs = [nlp(s) for s in data]

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 11.7 ms


In [8]:
def get_children_indicies(doc: spacy.tokens.doc.Doc):
    idxpairs = []
    for t in doc:
        idxpairs.extend([(c.i, t.i) for c in t.children])
    return idxpairs

deps_child_idx = [get_children_indicies(doc) for doc in docs]
deps_len = [len(doc) for doc in docs]

In [9]:
deps_child_idx[3]

[(0, 1), (3, 1), (9, 1), (2, 3), (4, 3), (6, 4), (5, 6), (8, 6), (7, 8)]

In [10]:
maxlen = 12
padded = pad_sequences_adjacency(deps_child_idx, deps_len, 
                                 maxlen=maxlen, padding='pre', truncating='pre')
padded[3]

[(2, 3), (5, 3), (11, 3), (4, 5), (6, 5), (8, 6), (7, 8), (10, 8), (9, 10)]

In [11]:
idx_pairs = padded[3]
n_seqlen = deps_len[3]

mat = torch.sparse.FloatTensor(
    torch.LongTensor(idx_pairs).t(),
    torch.FloatTensor([999 for _ in range(len(idx_pairs))]),
    torch.Size([maxlen, maxlen])
)
mat.to_dense()

tensor([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 999.,   0.,   0.,   0.],
        [  0.,   0.,   0., 999.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])